In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE60861"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE60861"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE60861.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE60861.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE60861.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll correct the code to properly handle the sample characteristics and extract clinical features.

```python
import pandas as pd
import os
import numpy as np
from typing import Dict, List, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to have both miRNA and mRNA data
# Since it mentions "renal miRNA- and mRNA-expression signatures", we can set is_gene_available to True
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (CKD progression status)
# Looking at the sample characteristics dictionary, we can see that key 4 contains 'clinical course: stable/progressive'
trait_row = 4

# For age
# Looking at the sample characteristics dictionary, keys 1 and 2 contain age data
# Key 1 has more consistent age data format, so we'll use it
age_row = 1

# For gender
# Looking at the sample characteristics dictionary, key 0 contains gender data
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert trait data to binary format:
    - 1 for progressive CKD
    - 0 for stable CKD
    - None for unknown/missing values
    """
    if pd.isna(value):
        return None
    
    value = value.lower().strip() if isinstance(value, str) else str(value).lower().strip()
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value == "progressive":
        return 1
    elif value == "stable":
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age data to continuous format.
    Extract numeric age value after the colon.
    """
    if pd.isna(value):
        return None
    
    value = str(value).strip()
    
    if ":" in value:
        age_str = value.split(":", 1)[1].strip()
        try:
            return float(age_str)
        except ValueError:
            return None
    else:
        try:
            return float(value)
        except ValueError:
            return None

def convert_gender(value):
    """
    Convert gender data to binary format:
    - 0 for female
    - 1 for male
    - None for unknown/missing values
    """
    if pd.isna(value):
        return None
    
    value = value.lower().strip() if isinstance(value, str) else str(value).lower().strip()
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value == "male":
        return 1
    elif value == "female":
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Create clinical_data DataFrame
    # The expected format for geo_select_clinical_features is a DataFrame where
    # each row corresponds to a feature category (gender, age, etc.)
    sample_chars_dict = {
        0: ['gender: male', 'gender: female', 'tissue: kidney biopsy'],
        1: ['age (yrs): 72', 'age (yrs): 20', 'age (yrs): 64', 'age (yrs): 17', 'age (yrs): 46', 'age (yrs): 55', 'age (yrs): 74', 'age (yrs): 49', 'age (yrs): 42', 'age (yrs): 73', 'age (yrs): 63', 'age (yrs): 33', 'age (yrs): 24', 'age (yrs): 45', 'age (yrs): 70', 'age (yrs): 60', 'age (yrs): 67', 'age (yrs): 31', 'age (yrs): 53', 'age (yrs): 22', 'age (yrs): 54', 'age (yrs): 40', 'age (yrs): 38', 'age (yrs): 19', 'age (yrs): 28', 'age (yrs): 65', 'age (yrs): 58', 'age (yrs): 56', 'age (yrs): 34', 'age (yrs): 59'],
        2: ['diagnosis: Diabetic Nephropathy', 'diagnosis: Focal-Segmental Glomerulosclerosis', 'diagnosis: Hypertensive Nephropathy', 'diagnosis: IgA-Nephropathy', 'diagnosis: Membranous Nephropathy', 'diagnosis: Minimal-Change Disease', 'diagnosis: Other/Unknown', 'age (yrs): 41.6', 'age (yrs): 59.0', 'age (yrs): 21.0', 'age (yrs): 33.0', 'age (yrs): 35.0', 'age (yrs): 24.0', 'age (yrs): 70.0', 'age (yrs): 43.0', 'age (yrs): 45.0', 'age (yrs): 44.0', 'age (yrs): 54.0', 'age (yrs): 74.0', 'age (yrs): 31.0', 'age (yrs): 49.0', 'age (yrs): 28.0', 'age (yrs): 26.0', 'age (yrs): 47.0', 'age (yrs): 20.0', 'age (yrs): 71.0', 'age (yrs): 58.0', 'age (yrs): 18.0', 'age (yrs): 32.0', 'age (yrs): 55.0'],
        3: ['clinical course: stable', 'clinical course: progressive', 'diagnosis: IgA nephropathy', 'diagnosis: Glomerulonephritis, not specified', 'diagnosis: Lupus nephritis class 4', 'diagnosis: Lupus nephritis class 5', 'diagnosis: Membranoproliferative glomerulonephritis', 'diagnosis: Focal-segmental glomerulosclerosis', 'diagnosis: Vasculitis', 'diagnosis: Membranous nephropathy', 'diagnosis: Lupus nephritis class 3', 'diagnosis: Minimal change disease', 'diagnosis: Diabetic nephropathy'],
        4: [np.nan, 'clinical course: progressive', 'clinical course: stable']
    }
    
    # Create a DataFrame with the correct structure for geo_select_clinical_features
    # Each row is a feature category, with the row index corresponding to the feature row numbers
    max_row = max(sample_chars_dict.keys()) + 1
    clinical_data = pd.DataFrame(index=range(max_row))
    
    # Add each feature's unique values as columns in the DataFrame
    for row_idx, values in sample_chars_dict.items():
        for col_idx, value in enumerate(values):
            clinical_data.loc[row_idx, col_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    preview_result = preview_df(selected_clinical_


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any
import gzip

# Get background information and clinical data from the dataset
background_info, clinical_data = get_background_and_clinical_data(in_cohort_dir)

# Get unique values for each row to understand the data structure
row_values = get_unique_values_by_row(clinical_data)

print("Background Information:")
for key, value in background_info.items():
    print(f"{key}: {value}")

print("\nSample Characteristics by Row:")
for row_idx, values in row_values.items():
    print(f"Row {row_idx}: {values}")

# 1. Gene Expression Data Availability
# Check if gene expression data is available based on background info
is_gene_available = True
if "platform_technology" in background_info:
    tech = background_info["platform_technology"].lower()
    if "mirna" in tech or "methylation" in tech:
        is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# Analyze the rows to identify trait, age, and gender information

# For trait (Chronic kidney disease)
trait_row = None
# Look for rows that might contain disease status information
for row_idx, values in row_values.items():
    values_str = ' '.join([str(v).lower() for v in values])
    if ('ckd' in values_str or 
        'chronic kidney disease' in values_str or 
        'kidney' in values_str or 
        'control' in values_str or 
        'case' in values_str or 
        'disease' in values_str or
        'patient' in values_str):
        trait_row = row_idx
        print(f"Found trait information in row {row_idx}: {values}")
        break

def convert_trait(value):
    if value is None:
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if ('ckd' in value or 
        'chronic kidney disease' in value or 
        'renal disease' in value or 
        'kidney disease' in value or 
        'patient' in value or 
        'case' in value):
        return 1
    elif ('control' in value or 
          'healthy' in value or 
          'normal' in value):
        return 0
    return None

# For age
age_row = None
# Look for rows that might contain age information
for row_idx, values in row_values.items():
    values_str = ' '.join([str(v).lower() for v in values])
    if 'age' in values_str:
        age_row = row_idx
        print(f"Found age information in row {row_idx}: {values}")
        break

def convert_age(value):
    if value is None:
        return None
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    import re
    matches = re.findall(r'\d+\.?\d*', value)
    if matches:
        try:
            return float(matches[0])
        except:
            return None
    return None

# For gender
gender_row = None
# Look for rows that might contain gender information
for row_idx, values in row_values.items():
    values_str = ' '.join([str(v).lower() for v in values])
    if 'gender' in values_str or 'sex' in values_str or 'male' in values_str or 'female' in values_str:
        gender_row = row_idx
        print(f"Found gender information in row {row_idx}: {values}")
        break

def convert_gender(value):
    if value is None:
        return None
    value = str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value or 'f' == value.strip():
        return 0
    elif 'male' in value or 'm' == value.strip():
        return 1
    return None

# Check if trait data is available
is_trait_available = trait_row is not None

# 3. Save Metadata
# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if is_trait_available:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("\nPreview of selected clinical features:")
    for key, values in preview.items():
        print(f"{key}: {values}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Cannot process clinical data: trait information not found in the dataset.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Assume gene data is available

# Extract gene data
try:
    # Extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # Print the first 20 gene/probe identifiers
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20].tolist())
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print(f"File path: {matrix_file}")
    print("Please check if the file exists and contains the expected markers.")


### Step 5: Gene Identifier Review

In [ ]:
# Examine the gene identifiers shown in the previous output
# These identifiers (like 'A_23_P100001') appear to be Agilent microarray probe IDs
# They are not standard human gene symbols and will need to be mapped

# Agilent probe IDs typically start with 'A_' followed by numbers and letters
# This format indicates these are probe identifiers, not gene symbols
# We will need to map these to standard gene symbols for analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Analyze the gene annotation dataframe to identify the correct columns for mapping
# Based on the previous output, we need:
# - 'ID' column which contains the probe identifiers matching those in gene_data
# - 'GENE_SYMBOL' column which contains the gene symbols we want to map to

# First get the gene expression data and annotation data again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_expression_data = get_genetic_data(matrix_file)
gene_annotation = get_gene_annotation(soft_file)

# Display the first few rows of gene annotation to confirm column selection
print("First few rows of gene annotation:")
print(gene_annotation[['ID', 'GENE_SYMBOL']].head())

# 2. Get a gene mapping dataframe
# Extract the relevant columns for mapping: probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"\nMapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_expression_data, mapping_df)
print(f"\nConverted gene expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nAfter normalization, gene data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# Save the gene data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Define the functions for extracting clinical features
def convert_trait(value):
    """Convert clinical course to binary format: 1 for progressive, 0 for stable"""
    if pd.isna(value):
        return None
    
    value = str(value).lower().strip()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'progressive':
        return 1
    elif value == 'stable':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age data to float"""
    if pd.isna(value):
        return None
    
    value = str(value).strip()
    if ':' in value:
        age_str = value.split(':', 1)[1].strip()
        try:
            return float(age_str)
        except ValueError:
            return None
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if pd.isna(value):
        return None
    
    value = str(value).lower().strip()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    return None

# Re-extract clinical data if the saved file doesn't exist
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
trait_row = 4  # Based on the analysis in step 2
age_row = 1    # Based on the analysis in step 2
gender_row = 0 # Based on the analysis in step 2

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)
print(f"Clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Load the normalized gene data (already done in step 7)
if 'gene_data' not in locals():
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data from {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values in the linked data
print("\nHandling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine if the trait and demographic features are biased
print("\nChecking for bias in trait and demographic features...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation and save relevant information
print("\nConducting final quality validation...")
is_gene_available = True  # We've confirmed gene data is available in previous steps
is_trait_available = True  # We've confirmed trait data is available in previous steps

note = "This dataset contains gene expression data from kidney biopsies. It classifies samples based on clinical course (stable or progressive chronic kidney disease)."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved as dataset is not usable for the current trait study.")